In [6]:
#Oppdatert
import requests
import pandas as pd
import os
from datetime import datetime, timedelta

# Sett riktig mappe for lagring (opp ett nivå)
data_dir = "../data"
os.makedirs(data_dir, exist_ok=True)

API = "2a430097-ac9b-44e4-995e-a02afb9795b2"
SOURCE = "SN18700"
BASE_URL = "https://frost.met.no/observations/v0.jsonld"

start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)
delta = timedelta(days=30)

# Elementer som hentes i batcher (i én spørring)
elements_batch = {
    "mean(air_temperature P1D)": "Temperatur (°C)",
    "mean(air_pressure_at_sea_level P1D)": "Lufttrykk (hPa)",
    "sum(precipitation_amount P1D)": "Nedbør (mm)"
}

# Elementer som må hentes separat (på grunn av API-begrensninger)
elements_single = {
    "mean(relative_humidity P1D)": "Relativ fuktighet (%)",
    "mean(cloud_area_fraction P1D)": "Skydekke (oktas)"
}

force_update = input("Vil du hente nye data fra Frost API? (ja/nei): ").strip().lower()

# Funksjon for å hente data i batcher
def fetch_batch_elements():
    dataframes = {label: pd.DataFrame(columns=["Dato", label]) for label in elements_batch.values()}

    if all(os.path.exists(os.path.join(data_dir, f"{label}.csv")) for label in elements_batch.values()) and force_update != "ja":
        print("Bruker eksisterende filer for temperatur, trykk og nedbør.")
        return

    print("Henter temperatur, trykk og nedbør...")

    current_date = start_date
    while current_date <= end_date:
        batch_start = current_date.strftime("%Y-%m-%d")
        batch_end = min(current_date + delta, end_date).strftime("%Y-%m-%d")
        batch_reference_time = f"{batch_start}/{batch_end}"

        params = {
            "sources": SOURCE,
            "elements": ",".join(elements_batch.keys()),
            "referencetime": batch_reference_time
        }

        response = requests.get(BASE_URL, params=params, auth=(API, ""))
        
        if response.status_code == 200:
            raw_data = response.json().get("data", [])
            
            for entry in raw_data:
                dato = entry["referenceTime"].split("T")[0]
                if dato > end_date.strftime("%Y-%m-%d"):
                    continue
                for obs in entry["observations"]:
                    element_id = obs["elementId"]
                    verdi = obs["value"]
                    label = elements_batch[element_id]
                    df = dataframes[label]

                    df = pd.concat([df, pd.DataFrame([{"Dato": dato, label: verdi}])], ignore_index=True)
                    dataframes[label] = df
        else:
            print(f"Feil ved henting av data ({batch_reference_time}): {response.status_code}")
            print(response.text)

        current_date += delta

    for label, df in dataframes.items():
        df.drop_duplicates(subset="Dato", keep="last", inplace=True)
        df.sort_values("Dato", inplace=True)
        df.to_csv(os.path.join(data_dir, f"R_{label}.csv"), index=False, encoding="utf-8")
        print(f"Lagret til: {os.path.join(data_dir, f'R_{label}.csv')}")

# Funksjon for å hente enkeltstående elementer
def fetch_single_element(element_id, label):
    filename = os.path.join(data_dir, f"R_{label}.csv")
    if os.path.exists(filename) and force_update != "ja":
        print(f"Bruker eksisterende fil: {filename}")
        return

    print(f"Henter data for {label.lower()}...")
    params = {
        "sources": SOURCE,
        "elements": element_id,
        "referencetime": f"{start_date.strftime('%Y-%m-%d')}/{end_date.strftime('%Y-%m-%d')}"
    }

    response = requests.get(BASE_URL, params=params, auth=(API, ""))
    if response.status_code == 200:
        data = response.json().get("data", [])
        rows = [(d["referenceTime"].split("T")[0], d["observations"][0]["value"]) for d in data]
        pd.DataFrame(rows, columns=["Dato", label]).to_csv(filename, index=False, encoding="utf-8")
        print(f"{label} lagret som: {filename}")
    else:
        print(f"Feil ved henting av {label.lower()}: {response.status_code}\n{response.text}")

# Kjør begge deler
fetch_batch_elements()
for element_id, label in elements_single.items():
    fetch_single_element(element_id, label)


Henter temperatur, trykk og nedbør...


C:\Users\leahs\AppData\Local\Temp\ipykernel_29468\1160682549.py:71: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, pd.DataFrame([{"Dato": dato, label: verdi}])], ignore_index=True)


Lagret til: ../data\R_Temperatur (°C).csv
Lagret til: ../data\R_Lufttrykk (hPa).csv
Lagret til: ../data\R_Nedbør (mm).csv
Henter data for relativ fuktighet (%)...
Relativ fuktighet (%) lagret som: ../data\R_Relativ fuktighet (%).csv
Henter data for skydekke (oktas)...
Skydekke (oktas) lagret som: ../data\R_Skydekke (oktas).csv


# Kode som henter luftkvalitet fra API openwathermap:


In [4]:
import requests
import time
import csv
from datetime import datetime, timedelta
from collections import defaultdict
import os

data_dir = "../data"
os.makedirs(data_dir, exist_ok=True)

# --- Innstillinger ---
API_KEY = "e9ad2e06b3f1d6b100c9b20186bf202d" 
# Kordinater til Trondheim Torg
LAT = 63.4305 
LON = 10.3951

start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)

def to_unix(dt):
    return int(time.mktime(dt.timetuple()))

# --- Hent data ---
results = []

current_start = start_date
while current_start <= end_date:
    current_end = min(current_start + timedelta(days=5), end_date)
    url = (
        f"http://api.openweathermap.org/data/2.5/air_pollution/history"
        f"?lat={LAT}&lon={LON}"
        f"&start={to_unix(current_start)}&end={to_unix(current_end)}"
        f"&appid={API_KEY}"
    )

    print(f"Henter data: {current_start.date()} - {current_end.date()}")
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json().get("list", [])
        results.extend(data)
    else:
        print(f" Feil: {response.status_code} for {current_start.date()}")

    current_start = current_end + timedelta(days=1)
    time.sleep(1)

# --- Grupper og beregn snitt per dag ---
daily_data = defaultdict(lambda: {"pm2_5": [], "pm10": [], "no2": [], "o3": []})

for entry in results:
    dt = datetime.utcfromtimestamp(entry["dt"])
    if dt.date() < datetime(2023, 1, 1).date():
        continue  # Hopp over alt før 2023-01-01
    date_key = dt.date().isoformat()        
    components = entry["components"]
    for comp in ["pm2_5", "pm10", "no2", "o3"]:
        if comp in components:
            daily_data[date_key][comp].append(components[comp])


            
# --- Regn ut gjennomsnitt ---
aggregated_rows = []
for date, values in sorted(daily_data.items()):
    row = {"date": date}
    for comp, measurements in values.items():
        if measurements:
            row[comp] = round(sum(measurements) / len(measurements), 2)
        else:
            row[comp] = None
    aggregated_rows.append(row)

# --- Skriv til CSV ---
csv_file = os.path.join(data_dir, "R_luftkvalitet.csv")
fieldnames = ["date", "pm2_5", "pm10", "no2", "o3"]

with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(aggregated_rows)

print(f"\n Ferdig! CSV med daglig snitt lagret som: {csv_file}")


Henter data: 2023-01-01 - 2023-01-06
Henter data: 2023-01-07 - 2023-01-12
Henter data: 2023-01-13 - 2023-01-18
Henter data: 2023-01-19 - 2023-01-24
Henter data: 2023-01-25 - 2023-01-30
Henter data: 2023-01-31 - 2023-02-05
Henter data: 2023-02-06 - 2023-02-11
Henter data: 2023-02-12 - 2023-02-17
Henter data: 2023-02-18 - 2023-02-23
Henter data: 2023-02-24 - 2023-03-01
Henter data: 2023-03-02 - 2023-03-07
Henter data: 2023-03-08 - 2023-03-13
Henter data: 2023-03-14 - 2023-03-19
Henter data: 2023-03-20 - 2023-03-25
Henter data: 2023-03-26 - 2023-03-31
Henter data: 2023-04-01 - 2023-04-06
Henter data: 2023-04-07 - 2023-04-12
Henter data: 2023-04-13 - 2023-04-18
Henter data: 2023-04-19 - 2023-04-24
Henter data: 2023-04-25 - 2023-04-30
Henter data: 2023-05-01 - 2023-05-06
Henter data: 2023-05-07 - 2023-05-12
Henter data: 2023-05-13 - 2023-05-18
Henter data: 2023-05-19 - 2023-05-24
Henter data: 2023-05-25 - 2023-05-30
Henter data: 2023-05-31 - 2023-06-05
Henter data: 2023-06-06 - 2023-06-11
H